## Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering


## Loading Dataset

In [25]:
df = pd.read_csv(r'D:\Project Phase II\Dataset\phase2labelencodeddataset.csv',encoding='cp1252')

C:\Users\sathi\AppData\Local\Temp\ipykernel_14804\3370251438.py:1: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:\Project Phase II\Dataset\phase2labelencodeddataset.csv',encoding='cp1252')


## Preprocessing Data

In [28]:
#Find all string values in sport
df['sport'] = df['sport'].astype(str)
#Replace '-' with 0
df['sport'] = df['sport'].replace('-', '80')
#COnvert hexadecimal values to decimal in sport
df['sport'] = df['sport'].apply(lambda x: int(x, 16))
#Convert sport to int
df['sport'] = df['sport'].astype(int)

len(df['sport'].unique())

43005

In [27]:
df['dsport'] = df['dsport'].astype(str)
df['dsport'] = df['dsport'].replace('-', '80')
df['dsport'] = df['dsport'].apply(lambda x: int(x, 16))
df['dsport'] = df['dsport'].astype(int)
len(df['dsport'].unique())


64380

In [ ]:
dfAttack = df[df['Label']==1]
dfAttack.drop(['Label'], axis=1, inplace=True)
len(dfAttack)

C:\Users\sathi\AppData\Local\Temp\ipykernel_22772\4043762459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfAttack.drop(['Label'], axis=1, inplace=True)


321283

In [11]:
df

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.1,0,149.171.126.4,30306,114,5,0.119596,4550,68342,31,...,0,6,2,2,5,1,1,2,6,0
1,59.166.0.3,0,149.171.126.2,161545,114,5,0.650574,8928,320,31,...,0,3,5,2,4,1,1,4,6,0
2,59.166.0.8,0,149.171.126.9,20880,114,5,0.007980,2158,2464,31,...,0,3,5,1,1,1,1,3,6,0
3,149.171.126.18,0,175.45.176.3,83,120,6,0.000005,264,0,60,...,0,19,19,19,19,19,19,19,6,0
4,149.171.126.18,0,175.45.176.3,83,120,6,0.000005,264,0,60,...,0,19,19,19,19,19,19,19,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535847,59.166.0.5,0,149.171.126.7,0,114,5,0.087306,320,1828,31,...,0,1,2,3,3,1,1,3,6,0
2535848,59.166.0.7,0,149.171.126.4,0,114,2,0.365058,456,346,31,...,2,2,2,2,2,2,2,2,6,0
2535849,59.166.0.3,0,149.171.126.9,0,114,2,6.335154,1802,2088,31,...,2,2,2,4,2,2,2,2,6,0
2535850,59.166.0.9,0,149.171.126.0,0,114,2,2.200934,3498,166054,31,...,0,1,1,2,4,2,2,2,6,0


In [6]:
#Split train and test based on attack_cat
X = dfAttack.drop(['attack_cat'], axis=1)
y = dfAttack['attack_cat']
X = X.drop(['srcip','dstip'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Base Models

Logistic Regression

In [13]:
#Logistic regression to classify using all variables
from sklearn.metrics import roc_auc_score
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test,y_pred))
auc_scores = roc_auc_score(y_test, y_pred, multi_class='ovr')  # Or multi_class='ovo' for one-vs-one
print("AUC Scores for each class:", auc_scores)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       558
           1       0.00      0.00      0.00       445
           2       0.00      0.00      0.00      3355
           3       0.47      0.54      0.51      8946
           4       0.66      0.04      0.07      4897
           5       0.79      0.99      0.88     42913
           7       0.00      0.00      0.00      2807
           8       0.00      0.00      0.00       298
           9       0.00      0.00      0.00        38

    accuracy                           0.74     64257
   macro avg       0.21      0.17      0.16     64257
weighted avg       0.64      0.74      0.66     64257



AxisError: axis 1 is out of bounds for array of dimension 1

In [16]:
#Count the number of each attack category in XTrain
y_train.value_counts()

5    172568
3     35579
4     19349
2     12998
7     11180
0      2119
1      1884
8      1213
9       136
Name: attack_cat, dtype: int64

In [17]:
y_test.value_counts()

5    42913
3     8946
4     4897
2     3355
7     2807
0      558
1      445
8      298
9       38
Name: attack_cat, dtype: int64

In [15]:
#Training score of Logistic Regression
print("Training score of Logistic Regression: ", lr.score(X_train, y_train))
#For each class
print("Accuracy of Logistic Regression: ", accuracy_score(y_test, y_pred))

Training score of Logistic Regression:  0.7402947561725273
Accuracy of Logistic Regression:  0.7371025724823754


Decision Tree

In [18]:
#Decision Tree and also visualise the tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print(classification_report(y_test,y_pred))
print("Training score of Decision Tree: ", dt.score(X_train, y_train))
print("Accuracy of Decision Tree: ", accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.61      0.18      0.28       558
           1       0.49      0.10      0.17       445
           2       0.33      0.36      0.34      3355
           3       0.64      0.76      0.69      8946
           4       0.95      0.87      0.91      4897
           5       1.00      0.99      0.99     42913
           7       0.89      0.77      0.83      2807
           8       0.85      0.83      0.84       298
           9       0.55      0.55      0.55        38

    accuracy                           0.89     64257
   macro avg       0.70      0.60      0.62     64257
weighted avg       0.90      0.89      0.89     64257

Training score of Decision Tree:  0.9304973037747154
Accuracy of Decision Tree:  0.8899730768632211


In [26]:
#Sort the Gini Index and find the most important and least important features
#With the feature names

importances = dt.feature_importances_
indices = np.argsort(importances)[::-1]
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature %d %s (%f)" % (f + 1, indices[f], X.columns[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature 5 sbytes (0.622920)
2. feature 1 dsport (0.164337)
3. feature 11 service (0.037611)
4. feature 6 dbytes (0.021495)
5. feature 7 sttl (0.021233)
6. feature 21 dmeansz (0.017264)
7. feature 20 smeansz (0.016261)
8. feature 38 ct_srv_dst (0.012310)
9. feature 18 stcpb (0.011167)
10. feature 26 Stime (0.008336)
11. feature 9 sloss (0.005382)
12. feature 13 Dload (0.004789)
13. feature 37 ct_srv_src (0.004033)
14. feature 19 dtcpb (0.003968)
15. feature 31 synack (0.003931)
16. feature 32 ackdat (0.003838)
17. feature 12 Sload (0.003636)
18. feature 30 tcprtt (0.003404)
19. feature 4 dur (0.003202)
20. feature 25 Djit (0.003178)
21. feature 23 res_bdy_len (0.003127)
22. feature 28 Sintpkt (0.002834)
23. feature 24 Sjit (0.002834)
24. feature 27 Ltime (0.002691)
25. feature 29 Dintpkt (0.002579)
26. feature 10 dloss (0.002545)
27. feature 34 ct_flw_http_mthd (0.002181)
28. feature 40 ct_src_ ltm (0.001456)
29. feature 14 Spkts (0.001384)
30. feature 0 sport (0.001

## Dataset Sampling with Data Balancing

In [3]:
dfSample0 = df[df['Label'] == 0].sample(n=250000, random_state=42)
dfSample1 = df[df['Label'] == 1].sample(n=250000, random_state=42)

dfSample = pd.concat([dfSample0, dfSample1])
dfSample = dfSample.sample(frac=1, random_state=42).reset_index(drop=True)

#Split the dataset into features and labels
XSample = dfSample.drop('Label', axis=1)
ySample = dfSample['Label']

#Split the dataset into training and testing sets
XSample_train, XSample_test, ySample_train, ySample_test = train_test_split(XSample, ySample, test_size=0.2, random_state=42)

In [7]:
df1 = pd.read_csv(r'D:\Project Phase II\Dataset\completedataset.csv',encoding='cp1252')

C:\Users\sathi\AppData\Local\Temp\ipykernel_14804\3150562644.py:1: DtypeWarning: Columns (1,3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(r'D:\Project Phase II\Dataset\completedataset.csv',encoding='cp1252')


In [29]:
df['attack_cat'] = df.apply(lambda row: -1 if row['Label'] == 0 else row['attack_cat'], axis=1)


In [30]:
df['attack_cat'].value_counts()

-1    2214569
 5     215481
 3      44525
 4      24246
 2      16353
 7      13987
 0       2677
 1       2329
 8       1511
 9        174
Name: attack_cat, dtype: int64

In [31]:
dfLabel1 = df[df['Label'] == 1]

In [ ]:
['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd',
       'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat']

In [23]:
dfLabel1 = df1[df1['Label'] == 1]
dfLabel1 = dfLabel1.drop(['Stime','Ltime','srcip','dstip','sport','dsport'], axis = 1)

In [32]:
df['is_ftp_login'].value_counts()

0    2492277
1      43575
Name: is_ftp_login, dtype: int64

In [34]:
#Total number of unique records in the dataset
dfLabel1 = dfLabel1.drop_duplicates(['proto', 'state', 'dur', 'sbytes',
    'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
        'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
        'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
        'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd',
        ])

#'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm' all depend on 100 connections - Out of scope
len(dfLabel1)

58902

In [36]:
dfLabel0 = df[df['Label'] == 0]
#Total number of unique records in the dataset
dfLabel0 = dfLabel0.drop_duplicates(['proto', 'state', 'dur', 'sbytes',
    'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
        'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
        'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
        'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd',
        ])

#'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm' all depend on 100 connections - Out of scope
len(dfLabel0)

1641039

In [10]:
len(df)

2535852

In [37]:
dfLabel0.to_csv(r'D:\Project Phase II\Dataset\Label0.csv', index=False)
dfLabel1.to_csv(r'D:\Project Phase II\Dataset\Label1.csv', index=False)


In [42]:
dfSample0 = dfLabel0.sample(n=58902, random_state=2)

In [44]:
dfSample0['sttl'].value_counts()

31     57363
254      890
62       254
0        143
1         89
29        76
32        50
60        11
63         8
64         6
30         5
252        5
255        2
Name: sttl, dtype: int64

In [46]:
dfLabel0Train, dfLabel0Test = train_test_split(dfSample0, test_size=0.3,random_state=10)

In [47]:
dfLabel1Train, dfLabel1Test = train_test_split(dfLabel1, test_size=0.3,random_state=10)

In [48]:
dfLabel1Test

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
2203539,175.45.176.3,0,149.171.126.14,128,114,5,1.105683,68199,612,254,...,0,1,1,1,2,1,1,1,4,1
2133402,175.45.176.0,0,149.171.126.17,0,31,6,0.000001,180,0,254,...,0,4,3,3,4,4,3,3,3,1
1525985,175.45.176.3,0,149.171.126.14,1093,114,5,1.482214,1272,742,254,...,0,4,5,2,2,2,1,2,4,1
540949,175.45.176.0,0,149.171.126.15,128,114,5,0.671754,68181,612,254,...,0,3,3,10,1,1,1,3,4,1
1572234,175.45.176.1,0,149.171.126.13,0,114,5,0.820790,3048,834,254,...,0,4,4,1,1,1,1,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2613,175.45.176.1,0,149.171.126.10,377,114,5,0.741769,4678,354,254,...,0,4,3,1,1,1,1,3,4,1
1458847,175.45.176.1,0,149.171.126.11,377,114,5,1.201228,26576,642,254,...,0,4,4,1,1,1,1,1,4,1
1475832,175.45.176.0,2294806,149.171.126.19,32896,114,5,0.532201,2628,354,254,...,0,2,1,1,1,1,1,1,3,1
1379124,175.45.176.0,0,149.171.126.12,33,114,5,1.099823,872,682,254,...,1,2,2,1,1,1,1,2,4,1


In [49]:
train = pd.concat([dfLabel0Train, dfLabel1Train], ignore_index=True)
test = pd.concat([dfLabel0Test, dfLabel1Test], ignore_index=True)

In [50]:
train

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.9,0,149.171.126.8,83,120,2,0.001046,146,178,31,...,0,1,3,3,1,1,1,1,-1,0
1,59.166.0.0,0,149.171.126.4,0,114,5,1.005964,1580,10168,31,...,0,8,8,7,4,4,4,4,-1,0
2,59.166.0.7,0,149.171.126.5,0,114,5,0.042514,5174,90266,31,...,0,7,7,8,9,1,1,1,-1,0
3,59.166.0.9,0,149.171.126.0,0,114,5,7.320488,13454,548216,31,...,0,3,6,1,4,1,1,1,-1,0
4,59.166.0.1,0,149.171.126.5,0,114,5,5.814120,16744,687116,31,...,0,5,10,3,5,2,1,3,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82457,175.45.176.1,0,149.171.126.10,128,114,5,0.238637,744,5570,62,...,0,1,1,1,1,1,1,1,3,1
82458,175.45.176.2,0,149.171.126.16,1093,114,5,0.662554,1758,766,254,...,0,1,2,3,1,1,1,1,4,1
82459,175.45.176.0,0,149.171.126.10,1300,120,6,0.000010,632,0,254,...,0,5,5,2,2,2,1,6,4,1
82460,175.45.176.1,0,149.171.126.12,0,132,6,0.000004,200,0,254,...,0,21,16,6,6,6,6,6,4,1


In [52]:
test

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.9,0,149.171.126.0,86853,114,5,0.010884,2438,19502,31,...,0,6,2,4,2,1,1,2,-1,0
1,59.166.0.1,0,149.171.126.0,37,114,5,0.910855,37342,3380,31,...,0,1,2,1,1,1,1,3,-1,0
2,59.166.0.5,0,149.171.126.7,0,114,5,0.071288,3390,44988,31,...,0,3,2,1,3,1,1,1,-1,0
3,59.166.0.7,0,149.171.126.9,0,114,5,0.146954,2438,20194,31,...,0,12,5,4,3,1,1,1,-1,0
4,59.166.0.6,0,149.171.126.8,267044,120,2,0.001664,544,304,31,...,0,3,12,4,4,1,1,3,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35337,175.45.176.1,0,149.171.126.10,377,114,5,0.741769,4678,354,254,...,0,4,3,1,1,1,1,3,4,1
35338,175.45.176.1,0,149.171.126.11,377,114,5,1.201228,26576,642,254,...,0,4,4,1,1,1,1,1,4,1
35339,175.45.176.0,2294806,149.171.126.19,32896,114,5,0.532201,2628,354,254,...,0,2,1,1,1,1,1,1,3,1
35340,175.45.176.0,0,149.171.126.12,33,114,5,1.099823,872,682,254,...,1,2,2,1,1,1,1,2,4,1


In [59]:
len(test)

35342

In [51]:
train.to_csv(r'D:\Project Phase II\Dataset\train.csv', index=False)
test.to_csv(r'D:\Project Phase II\Dataset\test.csv', index=False)

In [11]:
dfSample0 = pd.read_csv(r'D:\Project Phase II\Dataset\dfSample0.csv',encoding='cp1252')
dfSample1 = pd.read_csv(r'D:\Project Phase II\Dataset\dfSample1.csv',encoding='cp1252')

dfSample = pd.concat([dfSample0, dfSample1])
dfSample = dfSample.sample(frac=1, random_state=42).reset_index(drop=True)

#Split the dataset into features and labels
XSample = dfSample.drop('Label', axis=1)
ySample = dfSample['Label']



C:\Users\sathi\AppData\Local\Temp\ipykernel_17560\2929915898.py:1: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfSample0 = pd.read_csv(r'D:\Project Phase II\Dataset\dfSample0.csv',encoding='cp1252')


In [14]:
XSample = XSample.drop(['srcip','dstip','sport','dsport', 'sttl', 'ct_state_ttl'], axis = 1 )
                        

In [15]:
#Split the dataset into training and testing sets
XSample_train, XSample_test, ySample_train, ySample_test = train_test_split(XSample, ySample, test_size=0.2, random_state=42)

In [5]:
dfSample0 

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.3,0,149.171.126.1,53,120,2,0.001048,130,162,31,...,0,4,3,2,2,2,1,1,6,0
1,10.40.85.30,0,10.40.85.1,0,6,2,31.081343,184,112,0,...,0,2,2,2,2,2,2,2,6,0
2,59.166.0.6,0,149.171.126.0,0,114,5,0.751008,1540,1644,31,...,0,11,5,5,6,4,1,4,6,0
3,59.166.0.4,0,149.171.126.1,0,114,5,0.004159,1064,2260,31,...,0,7,4,1,3,1,1,1,6,0
4,59.166.0.4,0,149.171.126.5,18529,114,5,0.005028,3928,2456,31,...,0,9,3,2,3,1,1,3,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,59.166.0.8,0,149.171.126.9,111,120,2,0.487342,568,320,31,...,0,7,4,3,3,1,1,3,6,0
249996,59.166.0.6,0,149.171.126.1,0,114,5,0.593803,37302,3276,31,...,0,2,2,4,2,1,1,1,6,0
249997,59.166.0.9,0,149.171.126.5,80,114,5,1.016747,1580,10168,31,...,0,2,3,3,1,1,1,2,6,0
249998,59.166.0.6,0,149.171.126.0,0,114,5,0.238773,320,1834,31,...,0,1,4,6,1,1,1,1,6,0


In [10]:
#Take equal number of samples for each label
dfDroppedforLR = df[df['Label'] == 0].sample(n=250000, random_state=40)
dfDroppedforLR = dfDroppedforLR.append(df[df['Label'] == 1].sample(n=250000, random_state=40))

#Drop srcip and dstip and use a new df
dfDroppedforLR = dfDroppedforLR.drop(['srcip', 'dstip','attack_cat','Stime','Ltime'], axis=1)

#Shuffle the dataset
dfDroppedforLR = dfDroppedforLR.sample(frac=1).reset_index(drop=True)

#Split the dataset into features and labels
X = dfDroppedforLR.iloc[:,0:42]
y = dfDroppedforLR['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

C:\Users\sathi\AppData\Local\Temp\ipykernel_5932\3429069213.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfDroppedforLR = dfDroppedforLR.append(df[df['Label'] == 1].sample(n=250000, random_state=40))


In [22]:
#Drop sttl and ct_state_ttl
X_train = X_train.drop(['sttl','ct_state_ttl'], axis=1)
X_test = X_test.drop(['sttl','ct_state_ttl'], axis=1)


KeyError: "['sttl', 'ct_state_ttl'] not found in axis"

In [11]:
#COnvert sport and dsport to int
X_train['sport'] = X_train['sport'].astype(str)
X_train['sport'] = X_train['sport'].replace('-', '80')
X_train['sport'] = X_train['sport'].apply(lambda x: int(x, 16))
X_train['sport'] = X_train['sport'].astype(int)

X_train['dsport'] = X_train['dsport'].astype(str)
X_train['dsport'] = X_train['dsport'].replace('-', '80')
X_train['dsport'] = X_train['dsport'].apply(lambda x: int(x, 16))
X_train['dsport'] = X_train['dsport'].astype(int)

X_test['sport'] = X_test['sport'].astype(str)
X_test['sport'] = X_test['sport'].replace('-', '80')
X_test['sport'] = X_test['sport'].apply(lambda x: int(x, 16))
X_test['sport'] = X_test['sport'].astype(int)

X_test['dsport'] = X_test['dsport'].astype(str)
X_test['dsport'] = X_test['dsport'].replace('-', '80')
X_test['dsport'] = X_test['dsport'].apply(lambda x: int(x, 16))
X_test['dsport'] = X_test['dsport'].astype(int)

NameError: name 'X_train' is not defined

## Baseline Models with Sampled Dataset

Decision Tree

In [16]:
#Decision Tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print(classification_report(y_test,y_pred))
print("Training score of Decision Tree: ", dt.score(X_train, y_train))
print("Accuracy of Decision Tree: ", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     50081
           1       0.99      0.99      0.99     49919

    accuracy                           0.99    100000
   macro avg       0.99      0.99      0.99    100000
weighted avg       0.99      0.99      0.99    100000

Training score of Decision Tree:  0.9998425
Accuracy of Decision Tree:  0.99009


In [19]:
# Find which records are not predicted correctly by decision tree
#Construct the test dataset
dfTest = X_test 
dfTest['Label'] = y_test
incorrect_predictions = dfTest[dfTest['Label'] != y_pred]


In [20]:
incorrect_predictions

,sport,dsport,proto,state,dur,sbytes,dbytes,dttl,sloss,dloss,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
472236,0,377,114,5,0.850604,534,354,252,2,1,...,0,0,3,3,2,2,1,1,3,1
135080,0,313,114,5,1.299769,1378,354,252,2,1,...,0,0,1,1,1,1,1,1,1,1
234332,0,1124,120,6,0.000009,89,0,0,0,0,...,0,0,6,1,1,3,1,1,1,1
88565,0,1093,114,5,0.314702,654,268,252,2,1,...,0,0,2,2,2,2,2,1,2,1
23854,0,13062,114,5,0.890001,608,646,252,2,3,...,0,0,2,2,1,1,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,0,377,114,5,0.712601,534,268,252,2,1,...,0,0,4,2,1,5,1,1,2,1
469749,0,1093,114,5,1.855134,994,860,252,3,2,...,0,0,1,1,1,2,1,1,1,1
495035,0,1364,120,6,0.000009,746,0,0,0,0,...,0,0,2,1,1,1,1,1,1,0
23138,0,33,114,5,2.135790,810,682,252,4,4,...,1,1,2,1,1,1,1,1,1,1


In [21]:
importances = dt.feature_importances_
indices = np.argsort(importances)[::-1]
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature %d %s (%f)" % (f + 1, indices[f], X.columns[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature 12 Sload (0.656645)
2. feature 19 dtcpb (0.131808)
3. feature 7 sttl (0.081627)
4. feature 35 ct_srv_src (0.053640)
5. feature 14 Spkts (0.013224)
6. feature 5 sbytes (0.009454)
7. feature 25 Djit (0.008395)
8. feature 20 smeansz (0.007258)
9. feature 1 dsport (0.006568)
10. feature 3 state (0.004796)
11. feature 34 ct_ftp_cmd (0.003783)
12. feature 6 dbytes (0.002595)
13. feature 27 Dintpkt (0.001698)
14. feature 11 service (0.001685)
15. feature 28 tcprtt (0.001626)
16. feature 18 stcpb (0.001533)
17. feature 33 is_ftp_login (0.001465)
18. feature 17 dwin (0.001426)
19. feature 15 Dpkts (0.001231)
20. feature 29 synack (0.001204)
21. feature 26 Sintpkt (0.001196)
22. feature 4 dur (0.001143)
23. feature 39 ct_src_dport_ltm (0.000951)
24. feature 10 dloss (0.000833)
25. feature 24 Sjit (0.000816)
26. feature 36 ct_srv_dst (0.000781)
27. feature 23 res_bdy_len (0.000752)
28. feature 37 ct_dst_ltm (0.000388)
29. feature 9 sloss (0.000330)
30. feature 0 sport 

IndexError: index 40 is out of bounds for axis 0 with size 40

In [29]:
#Sort the Gini Index and find the most important and least important features
#With the feature names

importances = dt.feature_importances_
indices = np.argsort(importances)[::-1]
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature %d %s (%f)" % (f + 1, indices[f], X.columns[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature 31 ct_state_ttl (0.963163)
2. feature 1 dsport (0.005711)
3. feature 7 sttl (0.004860)
4. feature 20 smeansz (0.003025)
5. feature 5 sbytes (0.001611)
6. feature 19 dtcpb (0.001538)
7. feature 12 Sload (0.001507)
8. feature 35 ct_srv_src (0.001440)
9. feature 18 stcpb (0.001399)
10. feature 36 ct_srv_dst (0.001372)
11. feature 30 ackdat (0.001175)
12. feature 29 synack (0.001172)
13. feature 4 dur (0.001125)
14. feature 28 tcprtt (0.001073)
15. feature 25 Djit (0.001035)
16. feature 41 ct_dst_src_ltm (0.000907)
17. feature 26 Sintpkt (0.000867)
18. feature 24 Sjit (0.000838)
19. feature 27 Dintpkt (0.000773)
20. feature 38 ct_src_ ltm (0.000751)
21. feature 16 swin (0.000649)
22. feature 37 ct_dst_ltm (0.000606)
23. feature 13 Dload (0.000524)
24. feature 23 res_bdy_len (0.000513)
25. feature 39 ct_src_dport_ltm (0.000491)
26. feature 6 dbytes (0.000373)
27. feature 21 dmeansz (0.000368)
28. feature 0 sport (0.000333)
29. feature 40 ct_dst_sport_ltm (0.00023

PCA for sttl and ct_state_ttl with Logistic Regression

In [30]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
X = X[['sttl', 'ct_state_ttl']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pca_x_train = pca.fit_transform(X_train)
pca_x_test = pca.fit_transform(X_test)
classifier = LogisticRegression()
classifier.fit(pca_x_train, y_train)
y_pred = classifier.predict(pca_x_test)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.93      0.98      0.96     49981
           1       0.98      0.93      0.96     50019

    accuracy                           0.96    100000
   macro avg       0.96      0.96      0.96    100000
weighted avg       0.96      0.96      0.96    100000



In [33]:
#Linear equation formed by PCA
print("PCA Co efficients using Linear Equation of Logistic Regression: ", classifier.coef_)
print("PCA Co efficients for sttl and ct_state_ttl:")
print(pca.components_)

PCA Co efficients using Linear Equation of Logistic Regression:  [[-0.03420537]]
PCA Co efficients for sttl and ct_state_ttl:
[[-0.99996548 -0.00830924]]


PCA for sttl and ct_state_ttl with Decision Tree

In [44]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
X = X[['ct_state_ttl','sttl']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pca_x_train = pca.fit_transform(X_train)
pca_x_test = pca.fit_transform(X_test)
classifier = DecisionTreeClassifier()
classifier.fit(pca_x_train, y_train)
y_pred = classifier.predict(pca_x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     49981
           1       0.98      1.00      0.99     50019

    accuracy                           0.99    100000
   macro avg       0.99      0.99      0.99    100000
weighted avg       0.99      0.99      0.99    100000



In [45]:
#Linear equation formed by PCA
print("PCA Co efficients for sttl and ct_state_ttl:")
print(pca.components_)

PCA Co efficients for sttl and ct_state_ttl:
[[-0.00830924 -0.99996548]]


## Reducing Baseline Model Performance

Dropping sttl and ct_state_ttl

In [6]:
XSample_train = XSample_train.drop(['sttl', 'ct_state_ttl'], axis = 1)
XSample_test =  XSample_test.drop(['sttl', 'ct_state_ttl'], axis = 1)


In [9]:
XSample_train = XSample_train.drop(['Stime', 'Ltime', 'srcip', 'dstip'], axis = 1)
XSample_test =  XSample_test.drop(['Stime', 'Ltime', 'srcip', 'dstip'], axis = 1)

In [13]:
XSample_train = XSample_train.drop(['sport', 'dsport'], axis = 1)
XSample_test =  XSample_test.drop(['sport', 'dsport'], axis = 1)

In [7]:
XSample_train

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,dttl,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat
269056,175.45.176.0,0,149.171.126.19,135,114,5,0.636964,1612,268,252,...,0,0,7,7,2,2,2,1,7,6
499174,59.166.0.9,0,149.171.126.7,0,114,5,0.012723,2646,23636,29,...,0,0,3,4,3,13,1,1,2,6
85143,59.166.0.1,0,149.171.126.3,60894,114,5,0.002848,320,1804,29,...,0,0,3,3,5,4,1,1,4,6
260335,59.166.0.3,0,149.171.126.6,0,114,5,0.034102,2230,14370,29,...,0,0,6,8,3,2,1,1,1,6
338124,59.166.0.6,0,149.171.126.9,0,120,2,0.001442,146,178,29,...,0,0,4,2,2,2,1,1,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,59.166.0.0,0,149.171.126.8,0,114,5,0.381023,1540,1644,29,...,0,0,14,10,9,16,6,1,7,6
365838,59.166.0.3,0,149.171.126.7,0,120,2,0.001103,146,178,29,...,0,0,10,6,7,9,3,3,3,6
131932,59.166.0.1,0,149.171.126.3,53,120,2,0.001051,146,178,29,...,0,0,1,1,3,2,1,1,2,6
146867,59.166.0.3,0,149.171.126.1,36483,114,5,0.061162,3806,54798,29,...,0,0,2,6,3,4,1,1,2,6


In [15]:
ySample_train.unique()

array([0], dtype=int64)

In [18]:

lrmodel = LogisticRegression()

# Train the model on the training data
lrmodel.fit(XSample_train, ySample_train)

# Make predictions on the testing data
lry_pred = lrmodel.predict(XSample_test)

report = classification_report(ySample_test,lry_pred)
print(report)

              precision    recall  f1-score   support

           0       0.92      0.77      0.84     49900
           1       0.81      0.94      0.87     50100

    accuracy                           0.86    100000
   macro avg       0.86      0.86      0.85    100000
weighted avg       0.86      0.86      0.85    100000



In [22]:
dtmodel = DecisionTreeClassifier()
dtmodel.fit(XSample_train, ySample_train)

dtmodel_pred = dtmodel.predict(XSample_test)

report = classification_report(ySample_test,dtmodel_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     49900
           1       1.00      1.00      1.00     50100

    accuracy                           1.00    100000
   macro avg       1.00      1.00      1.00    100000
weighted avg       1.00      1.00      1.00    100000



In [23]:
#KNN Classifier
knnmodel = KNeighborsClassifier()
knnmodel.fit(XSample_train, ySample_train)

knnmodel_pred = knnmodel.predict(XSample_test)

report = classification_report(ySample_test,knnmodel_pred)
print(report)

## LSTM

# Proper class-wise dataset sampling and splitting

In [9]:
dfsample0 = dfSample0.sample(n=12500, random_state=42)
dfsample1 = dfSample1.sample(n=12500, random_state=42)

dfsample0train, dfsample0test = train_test_split(dfsample0, test_size=0.2, random_state=42)
dfsample1train, dfsample1test = train_test_split(dfsample1, test_size=0.2, random_state=42)

dfsampletrain = pd.concat([dfsample0train, dfsample1train])
dfsampletest = pd.concat([dfsample0test, dfsample1test])

XSample_train = dfsampletrain[['srcip','sport','dstip','dsport','sttl', 'ct_state_ttl']]
ySample_train = dfsampletrain['Label']
XSample_test = dfsampletest[['srcip','sport','dstip','dsport','sttl', 'ct_state_ttl']]
ySample_test = dfsampletest['Label']



dfsample = dfsampletrain 

In [ ]:
dfsampleedge = 

# LSTM Dataset preprocessing for sttl and ct_state_ttl

In [10]:
nodes = set()
edges = set()
for i in range(len(dfsample)):
    src = str(dfsample['srcip'].iloc[i])+':'+str(dfsample['sport'].iloc[i])
    dst = str(dfsample['dstip'].iloc[i])+':'+str(dfsample['dsport'].iloc[i])
    nodes.add(src)
    nodes.add(dst)
    edges.add((src,dst))
len(nodes)

2995

In [42]:
len(edges)

3988

In [43]:
2995*2995

8970025

Training Dataset

In [11]:
import numpy as np
# Prepare an adjacency matrix of the nodes
adjacency_matrix = np.zeros((len(nodes), len(nodes)), dtype=float)
label_matrix = np.zeros((len(nodes), len(nodes)))

# Add the columns srcip:sport and dstip:dsport to the adjacency matrix
for i in range(len(dfsample)):
    src = str(dfsample['srcip'].iloc[i])+':'+str(dfsample['sport'].iloc[i])
    dst = str(dfsample['dstip'].iloc[i])+':'+str(dfsample['dsport'].iloc[i])
    src_index = list(nodes).index(src)
    dst_index = list(nodes).index(dst)
    adjacency_matrix[src_index, dst_index] = 0.99996548*dfsample['sttl'].iloc[i] + 0.00830924*dfsample['ct_state_ttl'].iloc[i]
    label_matrix[src_index, dst_index] = dfsample['Label'].iloc[i]
# Flatten tuple values in the adjacency matrix
adjacency_matrix_flat = adjacency_matrix.reshape(-1, 1)
label_flat = label_matrix.reshape(-1, 1) 


Testing Dataset

In [ ]:

nodes = set()
edges = set()
for i in range(len(dfsampletest)):
    src = str(dfsampletest['srcip'].iloc[i])+':'+str(dfsampletest['sport'].iloc[i])
    dst = str(dfsampletest['dstip'].iloc[i])+':'+str(dfsampletest['dsport'].iloc[i])
    nodes.add(src)
    nodes.add(dst)
    edges.add((src,dst))

print("Nodes:", len(nodes))
print("Testing records: ", len(edges))

In [ ]:
import numpy as np
# Prepare an adjacency matrix of the nodes
adjacency_matrix_test = np.zeros((len(nodes), len(nodes)), dtype=float)
label_matrix_test = np.zeros((len(nodes), len(nodes)))

# Add the columns srcip:sport and dstip:dsport to the adjacency matrix
for i in range(len(dfsampletest)):
    src = str(dfsampletest['srcip'].iloc[i])+':'+str(dfsampletest['sport'].iloc[i])
    dst = str(dfsampletest['dstip'].iloc[i])+':'+str(dfsampletest['dsport'].iloc[i])
    src_index = list(nodes).index(src)
    dst_index = list(nodes).index(dst)
    adjacency_matrix_test[src_index, dst_index] = 0.99996548*dfsampletest['sttl'].iloc[i] + 0.00830924*dfsampletest['ct_state_ttl'].iloc[i]
    label_matrix_test[src_index, dst_index] = dfsampletest['Label'].iloc[i]
# Flatten tuple values in the adjacency matrix
adjacency_matrix_test_flat = adjacency_matrix_test.reshape(-1, 1)
label_test_flat = label_matrix_test.reshape(-1, 1)

# LSTM MODEL TRAINING FOR edge_weight = 0.99996548 * sttl + 0.00830924 * ct_state_ttl (DONOT RUN)

In [49]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Modify the call method of the LSTMClassifier
class LSTMClassifier(tf.keras.Model):
    def __init__(self, input_shape, hidden_units, output_units):
        super(LSTMClassifier, self).__init__()
        self.lstm = LSTM(hidden_units, return_sequences=False)  # Set return_sequences=False since we only need the output at the last timestep
        self.dense = Dense(output_units, activation='sigmoid')

    def call(self, inputs):
        x = tf.expand_dims(inputs, axis=-1)  # Add a new dimension to represent the input features
        x = self.lstm(x)
        output = self.dense(x)
        return output



# Hyperparameters
input_shape = adjacency_matrix_flat.shape[1:]  # Assuming each tuple has 2 attributes
hidden_units = 16
output_units = 1  # Binary classification

# Convert adjacency_matrix_flat to a TensorFlow tensor
adjacency_tensor = tf.convert_to_tensor(adjacency_matrix_flat, dtype=tf.float32)

# Initialize model, optimizer, and loss function
model = LSTMClassifier(input_shape, hidden_units, output_units)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



# Training
history = model.fit(adjacency_tensor, label_flat, epochs=2, batch_size=32, validation_split=0.2)

# Evaluation
#loss, accuracy = model.evaluate(adjacency_tensor, label_flat) 
#print(f'Loss: {loss}, Accuracy: {accuracy}')





Epoch 1/2


224251/224251 [==============================] - 2646s 12ms/step - loss: 8.5013e-04 - accuracy: 1.0000 - val_loss: 1.1908e-05 - val_accuracy: 1.0000
Epoch 2/2
224251/224251 [==============================] - 2072s 9ms/step - loss: 5.4401e-05 - accuracy: 1.0000 - val_loss: 1.0904e-05 - val_accuracy: 1.0000


Saving the LSTM Model weights

In [51]:
# Save the model to HDF5 file
model.save_weights("lstm_weights_sttl_ct_state_ttl_pca.h5")

# Load the weights to create the LSTM for edge_weight = 0.99996548 * sttl + 0.00830924 * ct_state_ttl from stored data

LSTM needs some dummy data to get initialised

In [61]:
# Create an instance of the model
# Hyperparameters
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Modify the call method of the LSTMClassifier
class LSTMClassifier(tf.keras.Model):
    def __init__(self, input_shape, hidden_units, output_units):
        super(LSTMClassifier, self).__init__()
        self.lstm = LSTM(hidden_units, return_sequences=False)  # Set return_sequences=False since we only need the output at the last timestep
        self.dense = Dense(output_units, activation='sigmoid')

    def call(self, inputs):
        x = tf.expand_dims(inputs, axis=-1)  # Add a new dimension to represent the input features
        x = self.lstm(x)
        output = self.dense(x)
        return output
    
input_shape = adjacency_matrix_flat.shape[1:]  # Assuming each tuple has 2 attributes
hidden_units = 16
output_units = 1  # Binary classification
# Initialize model, optimizer, and loss function
model_test_load = LSTMClassifier(input_shape, hidden_units, output_units)

dummy_input = np.zeros((4, 4), dtype=float)
dummy_label = np.zeros((4, 4))
#Use dummy data to initialize the model
dummy_flat = dummy_input.reshape(-1, 1)
dummy_label_flat = dummy_label.reshape(-1, 1) 

model_test_load.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model_test_load.fit(dummy_flat, dummy_label_flat, epochs=2, batch_size=32, validation_split=0.2)


Epoch 1/2
1/1 [==============================] - 31s 31s/step - loss: 0.6931 - accuracy: 1.0000 - val_loss: 0.6921 - val_accuracy: 1.0000
Epoch 2/2
1/1 [==============================] - 0s 269ms/step - loss: 0.6921 - accuracy: 1.0000 - val_loss: 0.6910 - val_accuracy: 1.0000


In [63]:
# Load the saved weights
model_test_load.load_weights("lstm_weights_sttl_ct_state_ttl_pca.h5")

Preparing Testing dataset

In [19]:

nodes = set()
edges = set()
for i in range(len(dfsampletest)):
    src = str(dfsampletest['srcip'].iloc[i])+':'+str(dfsampletest['sport'].iloc[i])
    dst = str(dfsampletest['dstip'].iloc[i])+':'+str(dfsampletest['dsport'].iloc[i])
    nodes.add(src)
    nodes.add(dst)
    edges.add((src,dst))


In [65]:
len(nodes)

915

In [66]:
len(edges)

1422

In [20]:
import numpy as np
# Prepare an adjacency matrix of the nodes
adjacency_matrix_test = np.zeros((len(nodes), len(nodes)), dtype=float)
label_matrix_test = np.zeros((len(nodes), len(nodes)))

# Add the columns srcip:sport and dstip:dsport to the adjacency matrix
for i in range(len(dfsampletest)):
    src = str(dfsampletest['srcip'].iloc[i])+':'+str(dfsampletest['sport'].iloc[i])
    dst = str(dfsampletest['dstip'].iloc[i])+':'+str(dfsampletest['dsport'].iloc[i])
    src_index = list(nodes).index(src)
    dst_index = list(nodes).index(dst)
    adjacency_matrix_test[src_index, dst_index] = 0.99996548*dfsampletest['sttl'].iloc[i] + 0.00830924*dfsampletest['ct_state_ttl'].iloc[i]
    label_matrix_test[src_index, dst_index] = dfsampletest['Label'].iloc[i]
# Flatten tuple values in the adjacency matrix
adjacency_matrix_test_flat = adjacency_matrix_test.reshape(-1, 1)
label_test_flat = label_matrix_test.reshape(-1, 1)

In [68]:
predictions = model.predict(adjacency_matrix_test_flat)
predictionsclass = (predictions > 0.5).astype(int)

26164/26164 [==============================] - 146s 5ms/step


In [75]:
predictions_test = model_test_load.predict(adjacency_matrix_test_flat)
predictionsclass_test = (predictions_test > 0.5).astype(int)

25116/26164 [===========================>..] - ETA: 6s

In [ ]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(label_test_flat, predictionsclass_test)

print("Classification Report:")
print(report)

In [69]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(label_test_flat, predictionsclass)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    836839
         1.0       0.92      1.00      0.96       386

    accuracy                           1.00    837225
   macro avg       0.96      1.00      0.98    837225
weighted avg       1.00      1.00      1.00    837225



In [70]:
#AuC for each class
from sklearn.metrics import roc_auc_score
roc_auc_score(label_test_flat, predictions_test, multi_class='ovr')


0.999985708308815

In [73]:
auc_score = roc_auc_score(label_test_flat, predictions_test)

print("AUC Score for Positive Class (Label 1):", auc_score)

AUC Score for Positive Class (Label 1): 0.999985708308815


In [74]:
predictions_label_0 = 1 - predictions_test
labels_label_0 = 1 - label_test_flat

# Calculate AUC score for label = 0
auc_score_label_0 = roc_auc_score(labels_label_0, predictions_label_0)

print("AUC Score for Label 0:", auc_score_label_0)

AUC Score for Label 0: 0.999985708308815


# GRU

## TRAINING GRU FOR 0.99996548 * sttl + 0.00830924 * ct_state_ttl (DO NOT RUN)

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# Modify the call method of the GRUClassifier
class GRUClassifier(tf.keras.Model):
    def __init__(self, input_shape, hidden_units, output_units):
        super(GRUClassifier, self).__init__()
        self.gru = GRU(hidden_units, return_sequences=False)  # Set return_sequences=False since we only need the output at the last timestep
        self.dense = Dense(output_units, activation='sigmoid')

    def call(self, inputs):
        x = tf.expand_dims(inputs, axis=-1)  # Add a new dimension to represent the input features
        x = self.gru(x)
        output = self.dense(x)
        return output

# Hyperparameters
input_shape = adjacency_matrix_flat.shape[1:]  # Assuming each tuple has 2 attributes
hidden_units = 16
output_units = 1  # Binary classification

# Convert adjacency_matrix_flat to a TensorFlow tensor
adjacency_tensor = tf.convert_to_tensor(adjacency_matrix_flat, dtype=tf.float32)

# Initialize model, optimizer, and loss function
model = GRUClassifier(input_shape, hidden_units, output_units)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training
history = model.fit(adjacency_tensor, label_flat, epochs=2, batch_size=32, validation_split=0.2)





Epoch 1/2


224251/224251 [==============================] - 2353s 10ms/step - loss: 5.3099e-04 - accuracy: 1.0000 - val_loss: 1.0071e-04 - val_accuracy: 1.0000
Epoch 2/2
224251/224251 [==============================] - 2772s 12ms/step - loss: 4.6858e-05 - accuracy: 1.0000 - val_loss: 4.3394e-05 - val_accuracy: 1.0000


In [14]:
model.save_weights("gru_weights_sttl_ct_state_ttl_pca.h5")

In [21]:
predictions = model.predict(adjacency_matrix_test_flat)
predictionsclass = (predictions > 0.5).astype(int)

26164/26164 [==============================] - 243s 9ms/step


In [22]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(label_test_flat, predictionsclass)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    836839
         1.0       0.92      1.00      0.96       386

    accuracy                           1.00    837225
   macro avg       0.96      1.00      0.98    837225
weighted avg       1.00      1.00      1.00    837225



In [24]:
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(label_test_flat, predictions)

print("AUC Score for Positive Class (Label 1):", auc_score)

predictions_label_0 = 1 - predictions
labels_label_0 = 1 - label_test_flat

# Calculate AUC score for label = 0
auc_score_label_0 = roc_auc_score(labels_label_0, predictions_label_0)

print("AUC Score for Label 0:", auc_score_label_0)

AUC Score for Positive Class (Label 1): 0.9999865751285987
AUC Score for Label 0: 0.9999865751285988


## GRU FOR 0.99996548 * sttl + 0.00830924 * ct_state_ttl FROM STORED WEIGHTS

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# Modify the call method of the GRUClassifier
class GRUClassifier(tf.keras.Model):
    def __init__(self, input_shape, hidden_units, output_units):
        super(GRUClassifier, self).__init__()
        self.gru = GRU(hidden_units, return_sequences=False)  # Set return_sequences=False since we only need the output at the last timestep
        self.dense = Dense(output_units, activation='sigmoid')

    def call(self, inputs):
        x = tf.expand_dims(inputs, axis=-1)  # Add a new dimension to represent the input features
        x = self.gru(x)
        output = self.dense(x)
        return output

# Hyperparameters
input_shape = adjacency_matrix_flat.shape[1:]  # Assuming each tuple has 2 attributes
hidden_units = 16
output_units = 1  # Binary classification

model_test_load = GRUClassifier(input_shape, hidden_units, output_units)

dummy_input = np.zeros((4, 4), dtype=float)
dummy_label = np.zeros((4, 4))
#Use dummy data to initialize the model
dummy_flat = dummy_input.reshape(-1, 1)
dummy_label_flat = dummy_label.reshape(-1, 1) 

model_test_load.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model_test_load.fit(dummy_flat, dummy_label_flat, epochs=2, batch_size=32, validation_split=0.2)

# Convert adjacency_matrix_flat to a TensorFlow tensor


In [ ]:
# Load the saved weights
model_test_load.load_weights("gru_weights_sttl_ct_state_ttl_pca.h5")

In [25]:
predictions_test = model_test_load.predict(adjacency_matrix_test_flat)
predictionsclass_test = (predictions_test > 0.5).astype(int)

26164/26164 [==============================] - 227s 8ms/step


In [26]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(label_test_flat, predictionsclass_test)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    836839
         1.0       0.92      1.00      0.96       386

    accuracy                           1.00    837225
   macro avg       0.96      1.00      0.98    837225
weighted avg       1.00      1.00      1.00    837225



In [27]:
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(label_test_flat, predictions_test)

print("AUC Score for Positive Class (Label 1):", auc_score)

predictions_label_0 = 1 - predictions_test
labels_label_0 = 1 - label_test_flat

# Calculate AUC score for label = 0
auc_score_label_0 = roc_auc_score(labels_label_0, predictions_label_0)

print("AUC Score for Label 0:", auc_score_label_0)

AUC Score for Positive Class (Label 1): 0.9999865751285987
AUC Score for Label 0: 0.9999865751285988
